In [1]:
from openai import AsyncOpenAI
import asyncio
import numpy as np
import json
from itertools import combinations, chain
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, timedelta
api_key = open("api_key_unlimited.txt").read().strip()
client = AsyncOpenAI(api_key = api_key)

# response = await client.embeddings.create(input=["Hello, world!"] ,model = "text-embedding-3-large")

In [2]:
async def get_embedding(text, model="text-embedding-3-large"):
    if not text:
        return []
    if type(text) != list:
        response = await client.embeddings.create(input=[text], model=model)
        return response.data[0].embedding
    response = await client.embeddings.create(input=text, model=model)
    return [r.embedding for r in response.data]

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def flatten_values(values):
    for value in values:
        if isinstance(value, list):
            for item in value:
                yield item
        else:
            yield value

In [3]:
DEGREE_ALIAS = {
    "highschool": ["high_school", "high school", "highschool", "high-school", "secondary school", "secondaryschool", "secondary-school"],
    "associate": ["associate", "associates", "associate's", "associates'", "associate bachelor", "associate bs"],
    "bs": ["bachelor", "bachelors", "bachelor's", "bachelors'", "undergraduate", "undergrad", "undergrads", "undergrad's", "undergrads'", "bs", "ba", "b.sc", "b.sc.", "bsc", "bachelor of science", "bachelor of arts", "b.a.","b.a"],
    "ma": ["master", "masters", "master's", "masters'", "graduate", "grad", "grads", "grad's", "grads'", "ms", "ma", "m.sc", "m.sc.", "msc", "master of science", "master of arts", "m.a.","m.a"],
    "phd": ["phd", "ph.d", "doctorate", "doctoral", "doctor", "doctor's", "doctorate of philosophy", "doctorate of philosophy", "doctor of philosophy"],
    "md": ["md", "m.d", "doctor of medicine", "doctorate of medicine"],
    "do" : ["do", "d.o", "doctor of osteopathic medicine", "doctorate of osteopathic medicine"],
    "jd": ["jd", "j.d", "juris doctor", "juris doctorate"],
    "pharmd": ["pharmd", "pharm.d", "doctor of pharmacy", "doctorate of pharmacy"],
    "mba": ["mba", "m.b.a", "master of business administration"],
    "dba": ["dba", "d.b.a", "doctor of business administration"],
}
potential_degrees = list(DEGREE_ALIAS.keys())
potential_degrees_embeddings = await asyncio.gather(*[get_embedding(potential) for potential in potential_degrees])

for key in DEGREE_ALIAS:
    DEGREE_ALIAS[key] = sorted(DEGREE_ALIAS[key], key = len, reverse = True)
    

In [4]:
def string_from_education(education):
    string = ""
    if not education:
        return "null"
    for i in education:
        i["education_id"] = i["education_school_linkedin_url"] = i['education_school_logo'] = ''
        string += " ".join(value for value in flatten_values(i.values()))
        string += "."
    return string

def strings_from_education(education):
    strings = []
    for i in education:
        i["education_id"] = i["education_school_linkedin_url"] = i['education_school_logo'] = ''
        strings.append(" ".join(value for value in flatten_values(i.values())))
    return strings

def degree_normalization(degree):
    if not degree:
        return " "
    
    degree = degree.lower()
    for key, values in DEGREE_ALIAS.items():
        if any(value in degree for value in values):
            return key

    return degree

# helper function to calculate the similarity between two education entries
def calc_school_similarity(target, similar):
    if target['education_school'] == '' or similar['education_school'] == '':
        return 1
    return np.dot(target['education_school_embedding'], similar['education_school_embedding'])

def calc_degree_similarity(target, similar):
    return np.dot(target['education_degree_embedding'], similar['education_degree_embedding'])

async def calc_major_similarity(target, similar):
    if target['education_major'] in similar['education_major'] or similar['education_major'] in target['education_major']:
        return 1
    target_major_info = [target['education_major'], target['education_description'], target['education_activities']]
    similar_major_info = [similar['education_major'], similar['education_description'], similar['education_activities']]
    target_major_string = ''
    similar_major_string = ''
    for t,s in zip(target_major_info, similar_major_info):
        if t != '' and s != '':
            target_major_string += t
            target_major_string += ' '
            similar_major_string += s
            similar_major_string += ' '
    if target_major_string == '':
        return 1
    
    target_embedding, similar_embedding = await asyncio.gather(
        get_embedding(target_major_string), 
        get_embedding(similar_major_string)
    )

    return np.dot(target_embedding, similar_embedding)

def calc_time_similarity(target, similar):
    # initialize time similarity to 1 (and as for no all time stamps available)
    time_similarity = 1
    if target['education_startyear'] == '' or similar['education_startyear'] == '':
        return int(target['education_endyear'] == similar['education_endyear'])
    elif target['education_endyear'] == '' or similar['education_endyear'] == '':
        return int(target['education_startyear'] == similar['education_startyear'])

    # at this point we know that both startyear and endyear are available
    target_startyear = int(target['education_startyear'])
    target_endyear = int(target['education_endyear'])
    similar_startyear = int(similar['education_startyear'])
    similar_endyear = int(similar['education_endyear'])
    value = max(0, min(target_endyear, similar_endyear) - max(target_startyear, similar_startyear))
    target_duration = abs(target_endyear - target_startyear)
    similar_duration = abs(similar_endyear - similar_startyear)
    if target_duration == 0 and similar_duration == 0:
        time_similarity = int(target_startyear == similar_startyear)
    else:
        time_similarity = 2 * value / (target_duration + similar_duration)
    return time_similarity

# return a list of dictionaries of information (check variable result) for each education entry
async def get_weighted_embedding(education):
    async def process_single_education(e):
        result = {
            'education_embedding': 0, 
            'education_school_embedding': 0, 
            'education_degree_embedding': 0, 
            'education_major_embedding': 0, 
            'education_year_score': 0, 
            'education_string': '', 
            'education_school': '', 
            'education_degree': '', 
            'education_major': '', 
            'education_description': '',
            'education_activities': '',
            'education_startyear': '', 
            'education_endyear': ''
        }

        result['education_school'] = e['education_school']
        result['education_degree'] = e['education_degree_level'] if e['education_degree_level'] != "" else degree_normalization(e['education_degree'])
        result['education_major'] = e['education_major'] 
        result['education_description'] = e['education_description']
        result['education_activities'] = e['education_activities']
        result['education_startyear'] = e['education_startyear']
        result['education_endyear'] = e['education_endyear']
        result['education_string'] = string_from_education([e])

        # Parallelize the embedding fetching using asyncio.gather
        embeddings = await asyncio.gather(
            get_embedding(result['education_school']),
            get_embedding(result['education_degree']),
            get_embedding(result['education_major']),
            get_embedding(result['education_school'] + " " + result['education_degree'] + " " + result['education_major'])
        )

        result['education_school_embedding'] = embeddings[0]
        result['education_degree_embedding'] = embeddings[1]
        result['education_major_embedding'] = embeddings[2]
        result['education_embedding'] = embeddings[3]

        return result

    # Process each education entry concurrently
    results = await asyncio.gather(*[process_single_education(e) for e in education])
    
    return results

async def calculate_similarity_weighted(target_education, similar_education):
    # await asyncio.sleep(1)
    # return {"dummy": "dummy"}

    target_embedding, similar_embedding = await asyncio.gather(
            get_weighted_embedding(target_education),
            get_weighted_embedding(similar_education)
        )

    if not target_embedding or not similar_embedding:
        return 0
    
    target_similar_mapping = {}

    # Generate all combinations of target and similar embeddings
    all_combinations = [(t, s) for t in target_embedding for s in similar_embedding]
    
    combination_similarities = []
    for (t, s) in all_combinations:
        # Calculate school, degree, and time similarities synchronously
        school_similarity = calc_school_similarity(t, s)
        degree_similarity = calc_degree_similarity(t, s)
        time_similarity = calc_time_similarity(t, s)

        # Calculate major similarity asynchronously
        major_similarity = await calc_major_similarity(t, s)

        # Combine similarities and store in the list
        similarity_score = school_similarity * degree_similarity * major_similarity * time_similarity
        combination_similarities.append((t, s, similarity_score))
    
    while combination_similarities:
        # Find the pair with the maximum similarity score
        (t_max, s_max, max_similarity) = max(combination_similarities, key=lambda x: x[2])

        # Map the target to the most similar entry in the similar set
        target_similar_mapping[t_max['education_string']] = (s_max['education_string'], max_similarity)

        # Remove combinations involving the matched target and similar entries
        combination_similarities = [
            (t, s, sim) for t, s, sim in combination_similarities 
            if t != t_max and s != s_max
        ]

        # Recalculate the similarities for t_max and s_max
        time_similarity = calc_time_similarity(t_max, s_max)
        school_similarity = calc_school_similarity(t_max, s_max)
        degree_similarity = calc_degree_similarity(t_max, s_max)
        major_similarity = await calc_major_similarity(t_max, s_max)

        # Count the number of highly similar attributes
        num_match = (
            int(school_similarity > 0.9) + 
            int(degree_similarity > 0.9) + 
            int(major_similarity > 0.9)
        )
        print(num_match)
        
    return target_similar_mapping

In [5]:
async def get_position_info(position):
    async def process_single_position(pos):
        result = {
            'position_company_name': pos['position_company_name'] if pos['position_company_name'] else ' ',
            'position_end_date': 0,
            'position_location': pos['position_location'] if pos['position_location'] else ' ',
            'position_start_date': 0,
            'position_summary': pos['position_summary'] if pos['position_summary'] else ' ',
            'position_title': pos['position_title'] if pos['position_title'] else ' ',
        }

        if pos['position_end_date'] == 'present' or pos['position_end_date'] is None:
            result['position_end_date'] = datetime.now().date()
        elif pos['position_end_date'] == '':
            result['position_end_date'] = datetime.min.date()
        else:
            result['position_end_date'] = datetime.strptime(pos['position_end_date'], "%b %Y").date()
        
        if pos['position_start_date'] == '':
            result['position_start_date'] = datetime.min.date()
        elif pos['position_start_date'] is None:
            result['position_start_date'] = datetime.min.date()
        else:
            result['position_start_date'] = datetime.strptime(pos['position_start_date'], "%b %Y").date()
        
        (result['position_company_name_embedding'],
         result['position_location_embedding'],
         result['position_summary_embedding'],
         result['position_title_embedding']) = await asyncio.gather(
            get_embedding(result['position_company_name']),
            get_embedding(result['position_location']),
            get_embedding(result['position_summary']),
            get_embedding(result['position_title'])
        )
        return result
    
    results = await asyncio.gather(*[process_single_position(pos) for pos in position])
    return results

def calc_company_name_similarity(target, similar):
    if target['position_company_name'].strip() == '' or similar['position_company_name'].strip() == '':
        return 1
    return np.dot(target['position_company_name_embedding'], similar['position_company_name_embedding'])

def calc_location_similarity(target, similar):
    if target['position_location'].strip() == '' or similar['position_location'].strip() == '':
        return 1
    return np.dot(target['position_location_embedding'], similar['position_location_embedding'])

def calc_summary_similarity(target, similar):
    if target['position_summary'].strip() == '' or similar['position_summary'].strip() == '':
        return 1
    if target['position_summary'] in similar['position_summary'] or similar['position_summary'] in target['position_summary']:
        return 1
    return np.dot(target['position_summary_embedding'], similar['position_summary_embedding'])

def calc_title_similarity(target, similar):
    if target['position_title'].strip() == '' or similar['position_title'].strip() == '':
        return 1
    return np.dot(target['position_title_embedding'], similar['position_title_embedding'])

def calc_pos_time_similarity(target, similar):
    required_fields = ['position_start_date', 'position_end_date']
    
    for field in required_fields:
        if field not in target or field not in similar:
            return 0
        
    if target['position_start_date'] == ' ' or target['position_end_date'] == ' ' or similar['position_start_date'] == ' ' or similar['position_end_date'] == ' ':
        return 0
    elif target['position_start_date'] <= similar['position_start_date']:
        if target['position_end_date'] >= similar['position_end_date']:
            return 1
        elif (similar['position_end_date'] - target['position_end_date']) <= timedelta(days = 90):
            return 0.7
    else:
        if similar['position_end_date'] >= target['position_end_date']:
            return 1
        elif (target['position_end_date'] - similar['position_end_date']) <= timedelta(days = 90):
            return 0.7
    return 0

def get_custom_judge(target, similar):
    score = int(calc_company_name_similarity(target, similar) >= 0.67) + int(calc_location_similarity(target, similar) >= 0.75) + int(calc_summary_similarity(target, similar) >= 0.7) + int(calc_title_similarity(target, similar) >= 0.68)
    return score >= 3

async def calculate_position_weighted(target_position, similar_position):
    global total_match_num, perfect_match_num, target_similar_score_list
    target_position_list = await get_position_info(target_position)
    similar_position_list = await get_position_info(similar_position)

    # Remove positions with no time stamp
    # target_position_list = [x for x in target_position_list if x['position_start_date'] is not None and x['position_end_date'] is not None]
    # similar_position_list = [x for x in similar_position_list if x['position_start_date'] is not None and x['position_end_date'] is not None]
    
    target_position_list.sort(key = lambda x: x['position_end_date'])
    similar_position_list.sort(key = lambda x: x['position_end_date'])

    # Using -1 to denote unvisited target position
    target_similar_mapping = {target_position['position_title']: -1 for target_position in target_position_list}

    threshold = min(math.ceil(0.5 * len(similar_position_list)), math.ceil(len(target_position_list) * 0.5))

    if not target_position_list or not similar_position_list:
        return 0

    print(f"target position number: {len(target_position_list)} similar position number: {len(similar_position_list)}")
    print(f"initial threshold: {threshold}")

    while target_position_list and similar_position_list:
        target = target_position_list[0]
        similar = similar_position_list[0]

        if target['position_start_date'] == ' ' or target['position_end_date'] == ' ':
            target_position_list.pop(0)
            print(f"target position has no time stamp, remaining {len(target_position_list)} target positions and {len(similar_position_list)} similar positions")
            continue
        elif similar['position_start_date'] == ' ' or similar['position_end_date'] == ' ':
            similar_position_list.pop(0)
            print(f"similar position has no time stamp, remaining {len(target_position_list)} target positions and {len(similar_position_list)} similar positions")
            continue
        elif target['position_start_date'] <= similar['position_start_date'] and target['position_end_date'] >= similar['position_end_date']:
            similarity_score = np.prod([calc_company_name_similarity(target, similar), calc_location_similarity(target, similar), calc_summary_similarity(target, similar), calc_title_similarity(target, similar)])

            print(f"position time: {target['position_start_date']} - {target['position_end_date']} vs.\n {similar['position_start_date']} - {similar['position_end_date']}\n company name: {target['position_company_name']} vs. \n {similar['position_company_name']} similarity: {calc_company_name_similarity(target, similar)}\n location: {target['position_location']} vs. \n {similar['position_location']} similarity: {calc_location_similarity(target, similar)}\n summary: {target['position_summary']} vs. \n {similar['position_summary']} similarity: {calc_summary_similarity(target, similar)}\n title: {target['position_title']} vs. \n {similar['position_title']} similarity: {calc_title_similarity(target, similar)}\n overall similarity: {similarity_score}\n")

            judge = get_custom_judge(target, similar)
            if judge:
                print("This is a good match")
                threshold -= 1
                print(f"threshold value: {threshold}")
            else:
                print("This does not match")

            target_similar_mapping[target['position_title']] = (similar['position_title'], similarity_score)

        elif similar['position_start_date'] <= target['position_start_date'] and similar['position_end_date'] >= target['position_end_date']:
            similarity_score = np.prod([calc_company_name_similarity(target, similar), calc_location_similarity(target, similar), calc_summary_similarity(target, similar), calc_title_similarity(target, similar)])

            print(f"position time: {target['position_start_date']} - {target['position_end_date']} vs.\n {similar['position_start_date']} - {similar['position_end_date']}\n company name: {target['position_company_name']} vs. \n {similar['position_company_name']} similarity: {calc_company_name_similarity(target, similar)}\n location: {target['position_location']} vs. \n {similar['position_location']} similarity: {calc_location_similarity(target, similar)}\n summary: {target['position_summary']} vs. \n {similar['position_summary']} similarity: {calc_summary_similarity(target, similar)}\n title: {target['position_title']} vs. \n {similar['position_title']} similarity: {calc_title_similarity(target, similar)}\n overall similarity: {similarity_score}\n")

            judge = get_custom_judge(target, similar)
            if judge:
                print("This is a good match")
                threshold -= 1
                print(f"threshold value: {threshold}")
            else:
                print("This does not match")

            target_similar_mapping[target['position_title']] = (similar['position_title'], similarity_score)
        else:
            print(f"position time: {target['position_start_date']} - {target['position_end_date']} vs.\n {similar['position_start_date']} - {similar['position_end_date']}")

        if target['position_end_date'] < similar['position_end_date']:
            target_position_list.pop(0)
            print(f"target position ends earlier, remaining {len(target_position_list)} target positions and {len(similar_position_list)} similar positions")
        else:
            similar_position_list.pop(0)
            print(f"similar position ends earlier, remaining {len(target_position_list)} target positions and {len(similar_position_list)} similar positions")
    
        # print("=========================================================================================================")
    
    if threshold <= 0:
        perfect_match_num += 1
        print("threshold met, perfect match")
    else:
        print(f"threshold not met, remaining {threshold} positions")


    total_match_num += 1

    # Add a overall similarity analysis for the target position
    for target in target_similar_mapping:
        if target_similar_mapping[target] == -1:
            print(f"target position: {target} has no similar position")
            
    print("****************************************** next target similar pair ***************************************************************")
    return target_similar_mapping

async def calculate_position_all_comb(target, similar):
    # await asyncio.sleep(1)
    # return {"dummy": "dummy"}

    global total_match_num, perfect_match_num, target_similar_score_list, notmatch_num
    
    target_position_list, similar_position_list = await asyncio.gather(
        get_position_info(target), 
        get_position_info(similar)
    )

    if not target_position_list or not similar_position_list:
        return 0
    
    target_similar_mapping = {target_position['position_title']: -1 for target_position in target_position_list}

    threshold = min(math.ceil(0.5 * len(similar_position_list)), math.ceil(len(target_position_list) * 0.5))
    
    all_combinations = [(t, s) for t in target_position_list for s in similar_position_list]
    
    def calculate_combination_similarity(t, s):
        similarities = [
            calc_company_name_similarity(t, s),
            calc_location_similarity(t, s),
            calc_summary_similarity(t, s),
            calc_title_similarity(t, s),
            calc_pos_time_similarity(t, s)
        ]
        return t, s, np.sum(similarities), similarities
    
    combination_similarities = [calculate_combination_similarity(t, s) for t, s in all_combinations]
    
    while combination_similarities:
        (t_max, s_max, similarity, individual_similarities) = max(combination_similarities, key=lambda x: x[2])

        target_similar_mapping[t_max['position_title']] = (s_max['position_title'], similarity)
        combination_similarities = [(t, s, sim, inds) for t, s, sim, inds in combination_similarities if t != t_max and s != s_max]

        company_similarity = individual_similarities[0]
        location_similarity = individual_similarities[1]
        summary_similarity = individual_similarities[2]
        title_similarity = individual_similarities[3]
        time_similarity = individual_similarities[4]
     
        print(f"company name: {t_max['position_company_name']} vs. {s_max['position_company_name']} similarity: {individual_similarities[0]}\n"
              f"location: {t_max['position_location']} vs. {s_max['position_location']} similarity: {individual_similarities[1]}\n"
              f"summary: {t_max['position_summary']} vs. {s_max['position_summary']} similarity: {individual_similarities[2]}\n"
              f"title: {t_max['position_title']} vs. {s_max['position_title']} similarity: {individual_similarities[3]}\n"
              f"time similarity: {individual_similarities[4]}\n"
              f"overall similarity: {similarity}\n")
        
        total_similarity_score = sum(individual_similarities)
        target_similar_score_list.append(total_similarity_score)

        if total_similarity_score > 4:
            print("This is a perfect match!!")
            threshold -= 1
        elif total_similarity_score > 3:
            print("This is a good match")
            threshold -= 0.5

    if threshold <= 0:
        print("threshold met, perfect match")
        perfect_match_num += 1
    else:
        print(f"threshold not met, remaining {threshold} positions")
        notmatch_num += 1

    total_match_num += 1
    
    for target in target_similar_mapping:
        if target_similar_mapping[target] == -1:
            print(f"target position: {target} has no similar position")
            
    print("****************************************** next target similar pair ***************************************************************")

    return target_similar_mapping

In [6]:
data = []
with open('export_profiles.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))

print(data[0])

{'target': {'additional_info': {}, 'analytics': {'availability': 0, 'availability_reason': [0], 'business_administration_level': 'mba', 'compensation_base_ave': 84547, 'compensation_base_max': 109911, 'compensation_base_min': 59182, 'contact_quality': 'high', 'contact_ranking_tier': 2, 'current_company_info': {}, 'current_company_start_date': '2016-12-25 00:00:00', 'current_title_start_date': '2018-07-25 00:00:00', 'diversity_analytics': {}, 'education': 'bs', 'experience': '20+', 'focus': ['operations'], 'focus_score': {'operations': 0.99}, 'fulltime_years': 21.33150684931507, 'gender': 'male', 'has_contact': True, 'has_personal_email': True, 'healthcare': {}, 'industry': ['paper & forest products'], 'it_analytics': {}, 'norm_location': {'city': 'Stevens Point', 'continent': 'North America', 'country': 'United States', 'country_short': 'US', 'in_gdpr': False, 'latitude': 44.5235792, 'location_fmt': 'Stevens Point, Wisconsin, United States', 'location_value': 'Stevens Point, Wisconsin,

In [7]:
async def process(data, max_concurrent = 20):
    semaphore = asyncio.Semaphore(max_concurrent)

    async def sem_task(task):
        async with semaphore:
            return await task

    flattened_data_education = [(i['target']['education'], j['education']) for i in data for j in i['similar'].values()]
    flattened_data_position = [(i['target']['position'], j['position']) for i in data for j in i['similar'].values()]
    
    tasks_position = [sem_task(calculate_position_all_comb(target, similar)) for target, similar in flattened_data_position]
    tasks_education = [sem_task(calculate_similarity_weighted(target, similar)) for target, similar in flattened_data_education]

    results_position = await asyncio.gather(*tasks_position)
    results_education = await asyncio.gather(*tasks_education)
    
    # Combine results or process them further if needed
    results = {
        "positions": results_position,
        "education": results_education
    }
    
    return results

In [8]:
async def process_with_rate_limit(data, batch_size = 3000//8, delay = 60):
    flattened_data_education = [(i['target']['education'], j['education']) for i in data for j in i['similar'].values()]
    flattened_data_position = [(i['target']['position'], j['position']) for i in data for j in i['similar'].values()]
    total_batches = math.ceil(len(flattened_data_education) / batch_size)
    flattened_result = []
    for batch_num in range(total_batches):
        start = batch_num * batch_size
        end = min((batch_num + 1) * batch_size, len(flattened_data_education))
        batch_education = flattened_data_education[start:end]
        batch_position = flattened_data_position[start:end]
        flattened_result.extend(await asyncio.gather(*[calculate_position_all_comb(target, similar) for target, similar in batch_position]))
        flattened_result.extend(await asyncio.gather(*[calculate_similarity_weighted(target, similar) for target, similar in batch_education]))
        await asyncio.sleep(delay)
    return flattened_result

In [9]:
total_match_num, perfect_match_num, target_similar_score_list, notmatch_num = 0, 0, [], 0
for _ in tqdm(range(10)):
    result = await process(data)

  0%|          | 0/10 [00:00<?, ?it/s]

company name: Neenah Paper vs. Neenah Paper similarity: 0.99999899010347
location:   vs.   similarity: 1
summary: Managed planning and distribution operations.

Lead facility implementation of company-wide WMS system while delivering in excess of $400m in labor density improvements.  Implemented perpetual inventory control and process.  Improved ability to serve through improved resource and business execution. vs.  Managed planning and distribution operations.
 Lead facility implementation of company-wide WMS system while delivering in excess of $400m in labor density improvements.  Implemented perpetual inventory control and process.  Improved ability to serve through improved resource and business execution. similarity: 0.9866285589653593
title: Logistics Manager vs. Logistics Manager similarity: 1.0000001259372326
time similarity: 1
overall similarity: 4.986627675006062

This is a perfect match!!
company name: Neenah Paper vs. Neenah Paper similarity: 0.9999999940756159
location: W

  0%|          | 0/10 [00:08<?, ?it/s]


CancelledError: 